In [1]:
# Importing Libraries
import numpy as np
import glob
import cv2
import os
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import BatchNormalization
from sklearn.preprocessing import LabelEncoder

# Dataset Directories
train_dir = 'dataset/train/'
test_dir = 'dataset/test/'
test_img = 'test_image.png'

In [2]:
# Setting the Hyperparameters
batch_size = 64
epochs = 16

In [3]:
# Train Data
biodeg = []
for img in glob.glob(train_dir+'biodegradable/*.png'):
    n = cv2.imread(img)
    biodeg.append(n)


biodegtr = np.array(biodeg)
# print("Shape of Biodegradable Train Datas: ", biodegtr.shape)

nbiodeg = []
for img in glob.glob(train_dir+'nonbiodegradable/*.png'):
    n = cv2.imread(img)
    nbiodeg.append(n)

nbiodegtr = np.array(nbiodeg)
# print("Shape of Non-Biodegradable Train Datas: ", nbiodegtr.shape)

rrr = []
for img in glob.glob(train_dir+'rrrable/*.png'):
    n = cv2.imread(img)
    rrr.append(n)

rrrtr = np.array(rrr)
# print("Shape of Reduce/Reuse/Recyclable Train Datas: ", rrrtr.shape)

oth = []
for img in glob.glob(train_dir+'other/*.png'):
    n = cv2.imread(img)
    oth.append(n)

othtr = np.array(oth)
# print("Shape of Other Train Datas: ", othtr.shape)

x_train = np.concatenate((biodegtr, nbiodegtr, rrrtr, othtr), axis=0)

x_train = np.array(x_train)
print("Shape of Train Dataset: ", x_train.shape)
# x_train = x_train.reshape((x_train.shape[0], 50, 50, 3))

y_train = []
for folder in os.listdir(train_dir):
    f = os.path.join(train_dir, folder)
    for file in os.listdir(f):
        y_train.append(folder)
y_train = np.array(y_train)
# print(y_train.shape)

Shape of Train Dataset:  (4642, 50, 50, 3)


In [4]:
# Test Data
testbiodeg = []
for img in glob.glob(test_dir+'biodegradable/*.png'):
    n = cv2.imread(img)
    testbiodeg.append(n)

testbiodegte = np.array(testbiodeg)

testnbiodeg = []
for img in glob.glob(test_dir+'nonbiodegradable/*.png'):
    n = cv2.imread(img)
    testnbiodeg.append(n)

testnbiodegte = np.array(testnbiodeg)

testrrr = []
for img in glob.glob(test_dir+'rrrable/*.png'):
    n = cv2.imread(img)
    testrrr.append(n)

testrrrte = np.array(testrrr)

testoth = []
for img in glob.glob(test_dir+'other/*.png'):
    n = cv2.imread(img)
    testoth.append(n)

testothte = np.array(testoth)

x_test = np.concatenate(
    (testbiodegte, testnbiodegte, testrrrte, testothte), axis=0)

x_test = np.array(x_test)

print("Shape of Test Dataset: ", x_test.shape)
# x_test = x_test.reshape((x_test.shape[0], 50, 50, 3))

y_test = []
for folder in os.listdir(test_dir):
    f = os.path.join(test_dir, folder)
    for file in os.listdir(f):
        y_test.append(folder)
y_test = np.array(y_test)
# print(y_test.shape)

Shape of Test Dataset:  (1152, 50, 50, 3)


In [5]:
# Replace NULL values with median in x_train and x_test
x_train = np.nan_to_num(x_train, nan=np.nanmedian(x_train))
x_test = np.nan_to_num(x_test, nan=np.nanmedian(x_test))

In [6]:
# Label Encoding y_train and y_test
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [7]:
# creating model
classifier = Sequential()

classifier.add(Conv2D(
    10, (3, 3), activation='relu', input_shape=(50, 50, 3)))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(10, (3, 3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2)))


classifier.add(Conv2D(10, (3, 3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2)))


classifier.add(Flatten())

classifier.add(Dense(units=25, activation='relu'))
classifier.add(Dropout(0.2))

classifier.add(Dense(units=4, activation='softmax'))

classifier.compile(
    optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

classifier.fit(x_train, y_train, batch_size=batch_size,
               epochs=epochs, verbose=1, validation_data=(x_test, y_test))

Epoch 1/16
73/73 [==============================] - 3s 40ms/step - loss: 1.1797 - accuracy: 0.5228 - val_loss: 0.9710 - val_accuracy: 0.5877
Epoch 2/16
73/73 [==============================] - 3s 37ms/step - loss: 0.7782 - accuracy: 0.6874 - val_loss: 0.8669 - val_accuracy: 0.6528
Epoch 3/16
73/73 [==============================] - 3s 39ms/step - loss: 0.6981 - accuracy: 0.7227 - val_loss: 0.8898 - val_accuracy: 0.6510
Epoch 4/16
73/73 [==============================] - 3s 38ms/step - loss: 0.6364 - accuracy: 0.7477 - val_loss: 0.9160 - val_accuracy: 0.6780
Epoch 5/16
73/73 [==============================] - 3s 40ms/step - loss: 0.6059 - accuracy: 0.7613 - val_loss: 0.8750 - val_accuracy: 0.6623
Epoch 6/16
73/73 [==============================] - 3s 36ms/step - loss: 0.5931 - accuracy: 0.7592 - val_loss: 0.8953 - val_accuracy: 0.6814
Epoch 7/16
73/73 [==============================] - 3s 38ms/step - loss: 0.5706 - accuracy: 0.7783 - val_loss: 1.0594 - val_accuracy: 0.6832
Epoch 8/16
73

In [8]:
test = cv2.imread(test_img)
test = cv2.resize(test, (50, 50))
test = np.array(test)
test = np.expand_dims(test, axis=0)
print("Shape of Validation Data: ", test.shape)
pred = classifier.predict(test)

Shape of Validation Data:  (1, 50, 50, 3)
1/1 [==============================] - 0s 91ms/step


In [9]:
def pred_img(index):
    if index == 0:
        return 'Biodegradable'
    elif index == 1:
        return 'Non-Biodegradable'
    elif index == 3:
        return 'Reduce/Reuse/Recycle-able'
    else:
        return 'Others'


op = []
pred = np.array(pred.tolist())[0]
index = pred.tolist().index(max(pred))
# print(index)
print("Input Test Image falls under the Category:", pred_img(index))
print("Accuracy = ", max(pred))

Input Test Image falls under the Category: Non-Biodegradable
Accuracy =  0.9048671722412109
